# Second-level models

Simple models trained on meta-features, or predictions of first-level models.

1. simple model averaging (best performing, validation error 0.68)
2. linear model (penalised using Ridge regression, second-best performing, validation error 0.87)
3. gradient boosting with shallow trees (no significant improvement on single best model)

In [42]:
import numpy as np
import pandas as pd
import pickle
import os
from sklearn import linear_model
from sklearn import metrics
from catboost import CatBoostRegressor
from catboost import Pool
import itertools

In [10]:
DATA_FOLDER = '../Data/'
INTERM_RES_FOLDER = "../Intermediary results"
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))
with open(os.path.join(INTERM_RES_FOLDER, "sets_level2.p"), "rb") as f:
    X_level2,y_level2,months_level2 = pickle.load(f)
    
with open(os.path.join(INTERM_RES_FOLDER, "sets.p"), "rb") as f:
    Xall,yall = pickle.load(f)

## 1. Model averaging

In [7]:
weights_regr = np.linspace(0,1,51)
weights_knn = np.linspace(0,1,51)
weights_cat = np.linspace(0,1,51)
n_comb = 22907
results = pd.DataFrame({"w_regr": np.zeros(n_comb),
                        "w_knn": np.zeros(n_comb),
                        "w_cat": np.zeros(n_comb),
                        "w_nn": np.zeros(n_comb),
                        "MSE": np.zeros(n_comb)})
i = 0
for w1 in weights_regr:
    weights_knn_valid = weights_knn[weights_knn <= 1-w1]
    for w2 in weights_knn_valid:
        weights_cat_valid = weights_cat[weights_cat <= 1-w1-w2]
        for w3 in weights_cat_valid:
            w4 = 1-w1-w2-w3
            pred = X_level2.dot(np.r_[w1,w2,w3,w4])
            results.iloc[i,:4] = [w1,w2,w3,w4]
            results.iloc[i, 4] = metrics.mean_squared_error(y_pred=pred, y_true=y_level2.clip(0,20))
            i+=1

In [8]:
best_weights = results.iloc[results.MSE.argmin()]
print(best_weights)

w_regr    0.060000
w_knn     0.040000
w_cat     0.620000
w_nn      0.280000
MSE       0.685883
Name: 3882, dtype: float64


In [22]:
ytest = X_level2[months_level2 == 34].dot(np.r_[best_weights[:4]])
submission = Xall.loc[Xall.date_block_num == 34, ["shop_id","item_id"]]
submission["item_cnt_month"] = ytest
submission.loc[submission.shop_id == 11, "shop_id"] = 10
submission = test.merge(submission, how="left", on=["shop_id","item_id"])
submission.drop(columns=["shop_id","item_id"], inplace=True)
submission.to_csv("../Submissions/submission_convcomb.csv", index=False)# 0.946

## 2. Linear model

In [37]:
regularisation = np.linspace(0.2,3,15)
regr_level2_mse = pd.DataFrame({"alpha": regularisation, "MSE":np.zeros(len(regularisation))})

Xtrain = X_level2[months_level2 < 33]
ytrain = y_level2[months_level2 < 33]
Xtest = X_level2[months_level2 == 33]
ytest = y_level2[months_level2 == 33].clip(0, 20)

for i,reg in enumerate(regularisation):
    regr_level2 = linear_model.Ridge(normalize=True, alpha = reg)
    regr_level2.fit(Xtrain, ytrain)
    pred_level2 = regr_level2.predict(Xtest).clip(0,20)
    regr_level2_mse.at[i,"MSE"] = metrics.mean_squared_error(y_pred=pred_level2, y_true=ytest)
    
del Xtrain,ytrain,Xtest,ytest

In [38]:
regr_level2_mse

,alpha,MSE
0,0.2,1.030557
1,0.4,0.979330
2,0.6,0.943722
3,0.8,0.917798
4,1.0,0.898951
5,1.2,0.885676
6,1.4,0.876581
7,1.6,0.870799
8,1.8,0.867689
9,2.0,0.866625


In [39]:
best_alpha = regr_level2_mse.at[regr_level2_mse.MSE.argmin(),"alpha"]

In [40]:
ytest = regr_level2.predict(X_level2[months_level2 == 34]).clip(0,20)
submission = Xall.loc[Xall.date_block_num == 34, ["shop_id","item_id"]]
submission["item_cnt_month"] = ytest
submission.loc[submission.shop_id == 11, "shop_id"] = 10
submission = test.merge(submission, how="left", on=["shop_id","item_id"])
submission.drop(columns=["shop_id","item_id"], inplace=True)
submission.to_csv("../Submissions/submission_ensemble_lr.csv", index=False)# 0.965

## 3. Shallow gradient boosting

In [45]:
depth = [4,5,6]
regularisation = np.logspace(-2,1,4)
cat_level2_mse = pd.DataFrame({"depth": [i for i,j in itertools.product(depth,regularisation)],
                               "regul": [j for i,j in itertools.product(depth,regularisation)],
                               "MSE": np.zeros(len(depth)*len(regularisation))})

Xtrain = X_level2[months_level2 < 33]
ytrain = y_level2[months_level2 < 33]
Xtest = X_level2[months_level2 == 33]
ytest = y_level2[months_level2 == 33].clip(0,20)
pool_test = Pool(Xtest, ytest)

for d in depth:
    for reg in regularisation:
        cat = CatBoostRegressor(iterations=150,
                                learning_rate=0.05,
                                use_best_model=True,
                                loss_function="RMSE",
                                eval_metric="RMSE",
                                one_hot_max_size=2,
                                l2_leaf_reg=reg,
                                depth=d)
        cat.fit(Xtrain, ytrain, verbose=30, eval_set=pool_test)
        pred_level2 = cat.predict(Xtest).clip(0,20)
        cat_level2_mse.at[(cat_level2_mse.depth == d) & (cat_level2_mse.regul == reg),"MSE"]=\
            metrics.mean_squared_error(y_pred=pred_level2, y_true=ytest)
        
del Xtrain,ytrain,Xtest,ytest

0:	learn: 3.6522190	test: 1.1129986	best: 1.1129986 (0)	total: 179ms	remaining: 26.7s
30:	learn: 3.4251226	test: 1.0051996	best: 0.9866700 (20)	total: 4.13s	remaining: 15.8s
60:	learn: 3.3537548	test: 1.0694805	best: 0.9866700 (20)	total: 7.67s	remaining: 11.2s
90:	learn: 3.2981658	test: 1.1196561	best: 0.9866700 (20)	total: 11.4s	remaining: 7.41s
120:	learn: 3.2651029	test: 1.1500617	best: 0.9866700 (20)	total: 15s	remaining: 3.6s
149:	learn: 3.2268400	test: 1.1700053	best: 0.9866700 (20)	total: 18.5s	remaining: 0us

bestTest = 0.9866699733
bestIteration = 20

Shrink model to first 21 iterations.
0:	learn: 3.6523139	test: 1.1130002	best: 1.1130002 (0)	total: 117ms	remaining: 17.5s
30:	learn: 3.4254387	test: 1.0051348	best: 0.9866478 (20)	total: 3.49s	remaining: 13.4s
60:	learn: 3.3542365	test: 1.0694292	best: 0.9866478 (20)	total: 7.06s	remaining: 10.3s
90:	learn: 3.2986750	test: 1.1196346	best: 0.9866478 (20)	total: 10.7s	remaining: 6.92s
120:	learn: 3.2651810	test: 1.1513195	best: 0

In [46]:
cat_level2_mse

,depth,regul,MSE
0,4,0.01,0.961278
1,4,0.10,0.961270
2,4,1.00,0.968395
3,4,10.00,0.972286
4,5,0.01,0.957860
5,5,0.10,0.948085
6,5,1.00,0.959101
7,5,10.00,0.954850
8,6,0.01,0.946747
9,6,0.10,0.955517


In [ ]:
best_depth = 5
best_alpha = 0.1
best_iter = 18

In [ ]:

ytest = cat.predict(X_level2[months_level2 == 34]).clip(0,20)
submission = Xall.loc[Xall.date_block_num == 34, ["shop_id","item_id"]]
submission["item_cnt_month"] = ytest
submission.loc[submission.shop_id == 11, "shop_id"] = 10
submission = test.merge(submission, how="left", on=["shop_id","item_id"])
submission.drop(columns=["shop_id","item_id"], inplace=True)
submission.to_csv("../Submissions/submission_ensemble_cat.csv", index=False)